In [13]:
from utils.loader import FactoryLoader
from utils.ml import MLPipeline
from utils.preprocessing import PreprocessingFactory
from utils.feature_extraction import *
from utils.utils import *

VAL_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val"
TRAIN_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train"

In [14]:
percent = 100
random = False
batch_size = 24
th = 0.01

# Initialize the FactoryLoader
factory = PreprocessingFactory()
factory.gaussian_smoothing(5)
factory.clahe(clip_limit=1.5)
factory.pad2square(fill=np.nan)
factory.resize((150,150))
factory.hair_removal()
factory.normalize2float()

factory_loader = FactoryLoader(path=VAL_PATH, batch_size=batch_size, factory=factory, percentage=percent, shuffle=random)

# Create the feature extraction pipeline strategy and add desired features
strategy = FeatureExtractionStrategy()

strategy.add_extractor(GradientExtractor(threshold=th)) # Add gradient feature

strategy.add_extractor(ColorMomentsExtractor("rgb", threshold=th))   # Add color moments feature
strategy.add_extractor(ColorMomentsExtractor("lab", threshold=th))   # Add color moments feature
strategy.add_extractor(ColorMomentsExtractor("hsv", threshold=th))   # Add color moments feature

strategy.add_extractor(LBPExtractor(radius=1, n_points=8))
strategy.add_extractor(LBPExtractor(radius=2, n_points=16))
strategy.add_extractor(LBPExtractor(radius=3, n_points=24))
strategy.add_extractor(LBPExtractor(radius=4, n_points=32))

strategy.add_extractor(FourierTransformExtractor())

strategy.add_extractor(GLCMExtractor(properties=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation',]))

pipeline = MLPipeline(dataset_path=TRAIN_PATH, preprocessing_factory=factory, feature_strategy=strategy, classifiers=[], percentage=percent, verbose=True, shuffle=random, batch_size=batch_size)

INFO:utils.ml:MLPipeline initialized with dataset path: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train
INFO:utils.ml:Preprocessing steps


In [15]:
pipeline.run_feature_extraction()

INFO:utils.ml:Running feature extraction...


Processed 5/634 batches.
Processed 10/634 batches.
Processed 15/634 batches.
Processed 20/634 batches.
Processed 25/634 batches.
Processed 30/634 batches.
Processed 35/634 batches.
Processed 40/634 batches.
Processed 45/634 batches.
Processed 50/634 batches.
Processed 55/634 batches.
Processed 60/634 batches.
Processed 65/634 batches.
Processed 70/634 batches.
Processed 75/634 batches.
Processed 80/634 batches.
Processed 85/634 batches.
Processed 90/634 batches.
Processed 95/634 batches.
Processed 100/634 batches.
Processed 105/634 batches.
Processed 110/634 batches.
Processed 115/634 batches.
Processed 120/634 batches.
Processed 125/634 batches.
Processed 130/634 batches.
Processed 135/634 batches.
Processed 140/634 batches.
Processed 145/634 batches.
Processed 150/634 batches.
Processed 155/634 batches.
Processed 160/634 batches.
Processed 165/634 batches.
Processed 170/634 batches.
Processed 175/634 batches.
Processed 180/634 batches.
Processed 185/634 batches.
Processed 190/634 bat

INFO:utils.ml:Feature extraction completed. Extracted 15195 features.


Processed 634/634 batches.


In [16]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(n_estimators=100)
rf2 = RandomForestClassifier(n_estimators=150)
rf3 = RandomForestClassifier(n_estimators=250)

rf4 = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42) # Random Forest with Feature Selection via max_features

rf5 = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42) # Random Forest with Out-of-Bag Error (OOB)

rf6 = RandomForestClassifier(n_estimators=100, bootstrap=False, random_state=42) # Random Forest with Bootstrap Disabled

xgb1 = XGBClassifier(n_estimators=350)
xgb2 = XGBClassifier(n_estimators=450)
xgb3 = XGBClassifier(n_estimators=550)
xgb4 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=3, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb5 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=3, subsample=0.7, colsample_bytree=0.7)
xgb6 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8)
xgb7 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=0.1)
xgb8 = XGBClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb9 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb10 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb11 = XGBClassifier(learning_rate=0.1, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)

pipeline.classifiers = [rf1, rf2, rf3, rf4, rf5, rf6,# rf7, rf8, rf9,
                        # svm1, svm2, svm3, svm6, svm7, svm8,
                        xgb1, xgb2, xgb3, xgb4, xgb5, xgb6, xgb7, xgb8, xgb9, xgb10, xgb11]
pipeline.fitted_classifiers = {}

In [17]:
pipeline.fit_classifiers()

INFO:utils.ml:Fitting classifiers...
INFO:utils.ml:Fitting classifier: RandomForestClassifier0
INFO:utils.ml:Top 10 features for RandomForestClassifier0: [('lbp_rad2_bins16_1', 0.021093742237677127), ('color_moments_lab_A_mean', 0.018212618067019626), ('color_moments_hsv_S_min', 0.01798653075337035), ('lbp_rad3_bins24_1', 0.016993182217449013), ('lbp_rad1_bins8_4', 0.014992550956503536), ('color_moments_lab_B_skew', 0.014616912686399084), ('color_moments_lab_A_median', 0.013836841424519582), ('color_moments_hsv_H_max', 0.013786597510462774), ('color_moments_rgb_G_max', 0.012656045838924528), ('lbp_rad2_bins16_8', 0.01245498621866787)]
INFO:utils.ml:Fitted classifier: RandomForestClassifier0; Done in 13.552456617355347 seconds
INFO:utils.ml:Fitting classifier: RandomForestClassifier1
INFO:utils.ml:Top 10 features for RandomForestClassifier1: [('lbp_rad2_bins16_1', 0.021260015152415303), ('color_moments_lab_A_mean', 0.01629844183466191), ('color_moments_hsv_S_min', 0.01500826228873398), 

In [18]:
pipeline.predict_with_classifiers(VAL_PATH, percent)

INFO:utils.ml:Predicting with classifiers on dataset: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val


Processed 5/159 batches.
Processed 10/159 batches.
Processed 15/159 batches.
Processed 20/159 batches.
Processed 25/159 batches.
Processed 30/159 batches.
Processed 35/159 batches.
Processed 40/159 batches.
Processed 45/159 batches.
Processed 50/159 batches.
Processed 55/159 batches.
Processed 60/159 batches.
Processed 65/159 batches.
Processed 70/159 batches.
Processed 75/159 batches.
Processed 80/159 batches.
Processed 85/159 batches.
Processed 90/159 batches.
Processed 95/159 batches.
Processed 100/159 batches.
Processed 105/159 batches.
Processed 110/159 batches.
Processed 115/159 batches.
Processed 120/159 batches.
Processed 125/159 batches.
Processed 130/159 batches.
Processed 135/159 batches.
Processed 140/159 batches.
Processed 145/159 batches.
Processed 150/159 batches.
Processed 155/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier0
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier1


Processed 159/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier2
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier3
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier4
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier5
INFO:utils.ml:Predictions made with classifier: XGBClassifier6
INFO:utils.ml:Predictions made with classifier: XGBClassifier7
INFO:utils.ml:Predictions made with classifier: XGBClassifier8
INFO:utils.ml:Predictions made with classifier: XGBClassifier9
INFO:utils.ml:Predictions made with classifier: XGBClassifier10
INFO:utils.ml:Predictions made with classifier: XGBClassifier11
INFO:utils.ml:Predictions made with classifier: XGBClassifier12
INFO:utils.ml:Predictions made with classifier: XGBClassifier13
INFO:utils.ml:Predictions made with classifier: XGBClassifier14
INFO:utils.ml:Predictions made with classifier: XGBClassifier15
INFO:utils.ml:Predictions made with classifier: XGBClassifier16


{'GT': array([0, 0, 0, ..., 1, 1, 1]),
 'RandomForestClassifier0': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier1': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier2': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier3': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier4': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier5': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier6': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier7': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier8': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier9': array([0, 0, 1, ..., 1, 0, 1]),
 'XGBClassifier10': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier11': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier12': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier13': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier14': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier15': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier16': array([0, 0, 0, ..., 0, 0, 1])}

In [22]:
import pandas as pd

df = pd.DataFrame(data=pipeline.calculate_metrics())
df = df.T
df

INFO:utils.ml:Metrics for classifier GT: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'kappa': 1.0}
INFO:utils.ml:Metrics for classifier RandomForestClassifier0: {'accuracy': 0.8266596417281349, 'precision': 0.8266716116674238, 'recall': 0.8265155922211748, 'f1': 0.8265706259173933, 'kappa': 0.6531508850473039}
INFO:utils.ml:Metrics for classifier RandomForestClassifier1: {'accuracy': 0.8314014752370916, 'precision': 0.8314393239524278, 'recall': 0.8312405329719839, 'f1': 0.8313066485500554, 'kappa': 0.66262679060792}
INFO:utils.ml:Metrics for classifier RandomForestClassifier2: {'accuracy': 0.8285036880927292, 'precision': 0.8285026489595277, 'recall': 0.8283739411853726, 'f1': 0.8284216593149338, 'kappa': 0.6568502041516859}
INFO:utils.ml:Metrics for classifier RandomForestClassifier3: {'accuracy': 0.8261327713382508, 'precision': 0.826177720390239, 'recall': 0.8259610725526648, 'f1': 0.8260305842199291, 'kappa': 0.652077500161785}
INFO:utils.ml:Metrics for classifie

cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated

,accuracy,precision,recall,f1,kappa
GT,1.000000,1.000000,1.000000,1.000000,1.000000
RandomForestClassifier0,0.826660,0.826672,0.826516,0.826571,0.653151
RandomForestClassifier1,0.831401,0.831439,0.831241,0.831307,0.662627
RandomForestClassifier2,0.828504,0.828503,0.828374,0.828422,0.656850
RandomForestClassifier3,0.826133,0.826178,0.825961,0.826031,0.652078
RandomForestClassifier4,0.826133,0.826178,0.825961,0.826031,0.652078
RandomForestClassifier5,0.834826,0.834929,0.834625,0.834714,0.669452
XGBClassifier6,0.831928,0.832914,0.831429,0.831614,0.663453
XGBClassifier7,0.829821,0.830917,0.829293,0.829479,0.659214
XGBClassifier8,0.830084,0.831081,0.829579,0.829763,0.659757


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

highest = df[df["accuracy"] == df.iloc[1:]["accuracy"].max()]
idx = highest.index[0]

print(classification_report(pipeline.predictions["GT"], pipeline.predictions[idx]))
print(confusion_matrix(pipeline.predictions["GT"], pipeline.predictions[idx]))


              precision    recall  f1-score   support

           0       0.82      0.86      0.84      1931
           1       0.85      0.81      0.83      1865

    accuracy                           0.84      3796
   macro avg       0.84      0.83      0.84      3796
weighted avg       0.84      0.84      0.84      3796

[[1659  272]
 [ 353 1512]]
